In [1]:
import numpy as np


#加載數據集
def load_data():
    with open('決策樹數據.txt') as fr:
        lines = fr.readlines()

    x = np.empty((len(lines), 7), dtype=int)

    for i in range(len(lines)):
        line = lines[i].strip().split(',')
        x[i] = line

    test_x = x[10:]
    x = x[:10]

    return x, test_x


x, test_x = load_data()
x, test_x

(array([[0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 1, 1, 1, 0],
        [0, 2, 2, 0, 2, 1, 1],
        [2, 1, 1, 1, 0, 0, 1],
        [1, 1, 0, 0, 1, 1, 1],
        [2, 0, 0, 2, 2, 0, 1],
        [0, 0, 1, 1, 1, 0, 1]]),
 array([[0, 0, 1, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 1, 0, 1],
        [2, 2, 2, 2, 2, 0, 1],
        [2, 0, 0, 2, 2, 1, 1],
        [0, 1, 0, 1, 0, 0, 1]]))

In [2]:
#計算數據集的熵,當然這個熵是針對於y來說的
def get_entropy(_x):
    entropy = 0

    #統計y的熵就可以了
    y = _x[:, -1]

    #統計每個結果出現的次數,[8 9],表示0出現8次,1出現9次
    bincount = np.bincount(y)  #[8 9]
    for count in bincount:
        if count == 0:
            continue

        #出現次數 / 總次數 = 出現概率
        prob = count / len(_x)

        #熵 = p * log(p) * -1
        entropy -= prob * np.log2(prob)

    return entropy


get_entropy(x)

1.0

In [3]:
def get_gain(_x, col):
    #列熵
    col_entropy = 0

    #根據列值,把數據分成n份
    for value in set(_x[:, col]):
        x_by_col_and_value = _x[_x[:, col] == value]

        #這個數據子集出現的概率,很顯然,等於出現次數/總次數
        prob = len(x_by_col_and_value) / len(_x)

        #求數據子集的熵
        entropy = get_entropy(x_by_col_and_value)

        #這個列的熵,等於這個式子的累計
        col_entropy += prob * entropy

    #信息增益,就是切分數據後,熵值能下降多少,這個值越大越好
    gain = get_entropy(_x) - col_entropy

    #用這個就是id3決策樹,他傾向於選擇可取值多的列
    return gain


get_gain(x, 0)

0.2754887502163468

In [4]:
def get_split_col(_x):
    best_col = -1
    best_gain = 0

    #遍歷所有的列,最後一列是y,不需要計算
    for col in range(_x.shape[1] - 1):

        #信息增益,就是切分數據後,熵值能下降多少,這個值越大越好
        gain = get_gain(_x, col)

        #信息增益最大的列,就是要被拆分的列
        if gain > best_gain:
            best_gain = gain
            best_col = col

    return best_col


get_split_col(x)

0

In [5]:
#創建節點和葉子對象,用來構建樹
class Node():
    def __init__(self, col):
        self.col = col
        self.children = {}

    def __str__(self):
        return 'Node col=%d' % self.col


class Leaf():
    def __init__(self, y):
        self.y = y

    def __str__(self):
        return 'Leaf y=%d' % self.y


print(Node(0)), print(Leaf(1))

Node col=0
Leaf y=1


(None, None)

In [6]:
#打印樹的方法
def print_tree(node, prefix='', subfix=''):
    prefix += '-' * 4
    print(prefix, node, subfix)
    if isinstance(node, Leaf):
        return
    for i in node.children:
        subfix = 'value=' + str(i)
        print_tree(node.children[i], prefix, subfix)


print_tree(Node(0))

---- Node col=0 


In [7]:
#先在所有數據上求最大信息增益的列,結果是0
get_split_col(x)

0

In [8]:
#根據上面的結果,創建根節點,根節點根據列0的值來分割數據
root = Node(0)
print(root)

Node col=0


前剪枝：在構建決策樹的過程中，在每個結點處進行估計，若該結點的子樹不純度下降的估計不足以超過預先設定的閾值，則停止劃分，將該結點標記為葉子結點。

In [9]:
#判斷是否要分割節點
def pre_cut(_x, test_x):
    #先計算不分割前的測試正確率

    #取y
    _y = _x[:, -1]
    test_y = test_x[:, -1]

    #求眾數
    vote_y = np.bincount(_y).argmax()
    
    #計算測試正確率
    pre_correct = np.sum(test_y == vote_y)

    #求分割列
    split_col = get_split_col(_x)

    #計算分割後的測試正確率
    after_correct = 0
    for split_value in np.unique(_x[:, split_col]):

        #分割數據和測試數據
        sub_x = _x[_x[:, split_col] == split_value]
        sub_test_x = test_x[test_x[:, split_col] == split_value]

        #取y
        sub_y = sub_x[:, -1]
        sub_test_y = sub_test_x[:, -1]

        #求眾數
        sub_vote_y = np.bincount(sub_y).argmax()

        #計算測試正確率
        after_correct += np.sum(sub_test_y == sub_vote_y)

    #如果分割後的測試正確率提升了,則分割,否則不分割
    return pre_correct >= after_correct


pre_cut(x, test_x)

False

In [10]:
#添加子節點的方法
def create_children(_x, test_x, parent_node):

    #遍歷父節點col列所有的取值
    for split_value in np.unique(_x[:, parent_node.col]):

        #首先根據父節點col列的取值分割數據
        sub_x = _x[_x[:, parent_node.col] == split_value]

        sub_test_x = test_x[test_x[:, parent_node.col] == split_value]

        #取去重y值
        _y = sub_x[:, -1]

        #如果所有的y都是一樣的,說明是個葉子節點
        #如果分割後的測試正確率提升了,則分割,否則不分割
        if len(np.unique(_y)) == 1 or pre_cut(sub_x, sub_test_x):
            #求眾數
            vote_y = np.bincount(_y).argmax()
            parent_node.children[split_value] = Leaf(vote_y)
            continue

        #否則,是個分支節點,計算最佳切分列
        split_col = get_split_col(sub_x)

        #添加分支節點到父節點上
        parent_node.children[split_value] = Node(col=split_col)


create_children(x, test_x, root)

print_tree(root)

---- Node col=0 
-------- Leaf y=0 value=0
-------- Leaf y=0 value=1
-------- Leaf y=1 value=2


In [11]:
#預測方法,測試
def pred(_x, node):
    col_value = _x[node.col]
    node = node.children[col_value]

    if isinstance(node, Leaf):
        return node.y

    return pred(_x, node)


correct = 0
for i in x:
    if pred(i, root) == i[-1]:
        correct += 1

print(correct / len(x))

print('-------------------------')

correct = 0
for i in test_x:
    if pred(i, root) == i[-1]:
        correct += 1

print(correct / len(test_x))

0.7
-------------------------
0.5714285714285714
